In [1]:
# Tourism Experience Analytics - Data Cleaning and Preprocessing
# Step 2: Data Cleaning, Integration, and Feature Engineering

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("=" * 80)
print("TOURISM ANALYTICS - DATA CLEANING AND PREPROCESSING")
print("=" * 80)

# =============================================================================
# SECTION 1: DATA LOADING WITH PATH AUTO-DETECTION
# =============================================================================

print("\n1. LOADING DATA WITH AUTO-DETECTION")
print("-" * 50)

# Auto-detect correct path
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

possible_paths = ['data/raw/', '../data/raw/', '../../data/raw/', 'data\\raw\\', '..\\data\\raw\\']
correct_path = None

for path in possible_paths:
    if os.path.exists(path + 'Transaction.xlsx'):
        correct_path = path
        break

if correct_path is None:
    print("ERROR: Cannot locate data files. Please check your directory structure.")
    exit()

print(f"Using data path: {correct_path}")

# Load all datasets
datasets = {}
file_paths = {
    'transactions': 'Transaction.xlsx',
    'users': 'User.xlsx',
    'items': 'Updated_Item.xlsx',
    'cities': 'City.xlsx',
    'countries': 'Country.xlsx',
    'regions': 'Region.xlsx',
    'continents': 'Continent.xlsx',
    'attraction_types': 'Type.xlsx',
    'visit_modes': 'Mode.xlsx'
}

for name, filename in file_paths.items():
    try:
        df = pd.read_excel(correct_path + filename)
        datasets[name] = df
        print(f"✓ {name}: {df.shape[0]:,} rows")
    except Exception as e:
        print(f"❌ {name}: {str(e)}")

print(f"\nLoaded {len(datasets)} datasets successfully")

# =============================================================================
# SECTION 2: DATA QUALITY ASSESSMENT AND CLEANING
# =============================================================================

print("\n2. DATA QUALITY ASSESSMENT AND CLEANING")
print("-" * 50)

def clean_dataset(df, dataset_name, critical_columns=None):
    """Comprehensive data cleaning function"""
    print(f"\n📊 CLEANING {dataset_name.upper()}")
    print("-" * 40)
    
    original_shape = df.shape
    cleaning_log = []
    
    # 1. Check for duplicates
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        df = df.drop_duplicates()
        cleaning_log.append(f"Removed {duplicates} duplicate rows")
    
    # 2. Handle missing values
    missing_before = df.isnull().sum().sum()
    if missing_before > 0:
        print(f"Missing values found:")
        missing_summary = df.isnull().sum()[df.isnull().sum() > 0]
        for col, count in missing_summary.items():
            pct = (count / len(df)) * 100
            print(f"  {col}: {count} ({pct:.1f}%)")
        
        # Different strategies for different column types
        for col in df.columns:
            if df[col].isnull().any():
                if df[col].dtype in ['int64', 'float64']:
                    # Numerical columns - fill with median
                    df[col].fillna(df[col].median(), inplace=True)
                    cleaning_log.append(f"Filled {col} missing values with median")
                elif df[col].dtype == 'object':
                    # Categorical columns - fill with mode or 'Unknown'
                    mode_val = df[col].mode()
                    if len(mode_val) > 0:
                        df[col].fillna(mode_val.iloc[0], inplace=True)
                        cleaning_log.append(f"Filled {col} missing values with mode")
                    else:
                        df[col].fillna('Unknown', inplace=True)
                        cleaning_log.append(f"Filled {col} missing values with 'Unknown'")
    
    # 3. Data type optimization
    for col in df.columns:
        if df[col].dtype == 'int64':
            # Check if can be converted to smaller int type
            col_min, col_max = df[col].min(), df[col].max()
            if col_min >= 0:
                if col_max < 255:
                    df[col] = df[col].astype('uint8')
                elif col_max < 65535:
                    df[col] = df[col].astype('uint16')
                elif col_max < 4294967295:
                    df[col] = df[col].astype('uint32')
            else:
                if col_min >= -128 and col_max <= 127:
                    df[col] = df[col].astype('int8')
                elif col_min >= -32768 and col_max <= 32767:
                    df[col] = df[col].astype('int16')
    
    # 4. Remove invalid entries if critical columns specified
    if critical_columns:
        for col, (min_val, max_val) in critical_columns.items():
            if col in df.columns:
                before_count = len(df)
                df = df[(df[col] >= min_val) & (df[col] <= max_val)]
                removed = before_count - len(df)
                if removed > 0:
                    cleaning_log.append(f"Removed {removed} rows with invalid {col} values")
    
    # Print cleaning summary
    final_shape = df.shape
    print(f"Shape change: {original_shape} → {final_shape}")
    if cleaning_log:
        print("Cleaning actions taken:")
        for action in cleaning_log:
            print(f"  • {action}")
    else:
        print("No cleaning required - data was already clean!")
    
    return df

# Clean each dataset
if 'transactions' in datasets:
    # Define validation rules for transactions
    transaction_rules = {
        'Rating': (1, 5),
        'VisitMonth': (1, 12),
        'VisitYear': (2018, 2024)
    }
    datasets['transactions'] = clean_dataset(datasets['transactions'], 'transactions', transaction_rules)

for name, df in datasets.items():
    if name != 'transactions':  # Already cleaned above
        datasets[name] = clean_dataset(datasets[name], name)

# =============================================================================
# SECTION 3: DATA INTEGRATION AND MASTER DATASET CREATION
# =============================================================================

print("\n3. DATA INTEGRATION AND MASTER DATASET CREATION")
print("-" * 50)

if len(datasets) >= 4:  # Need at least main datasets
    print("Creating integrated master dataset...")
    
    # Start with transactions as base
    if 'transactions' in datasets:
        master_df = datasets['transactions'].copy()
        print(f"Starting with transactions: {len(master_df):,} records")
        
        # Join with users (geographic information)
        if 'users' in datasets:
            master_df = master_df.merge(datasets['users'], on='UserId', how='left')
            print(f"After joining users: {len(master_df):,} records")
        
        # Join with items (attraction information)
        if 'items' in datasets:
            master_df = master_df.merge(datasets['items'], on='AttractionId', how='left')
            print(f"After joining attractions: {len(master_df):,} records")
        
        # Join with reference tables
        reference_joins = [
            ('cities', 'CityId', 'CityId'),
            ('countries', 'CountryId', 'CountryId'),
            ('regions', 'RegionId', 'RegionId'),
            ('continents', 'ContinentId', 'ContinentId'),
            ('attraction_types', 'AttractionTypeId', 'AttractionTypeId'),
            ('visit_modes', 'VisitModeId', 'VisitMode')
        ]
        
        for ref_name, join_col, master_col in reference_joins:
            if ref_name in datasets and join_col in datasets[ref_name].columns:
                if master_col in master_df.columns:
                    before_cols = len(master_df.columns)
                    master_df = master_df.merge(
                        datasets[ref_name], 
                        left_on=master_col, 
                        right_on=join_col, 
                        how='left',
                        suffixes=('', f'_{ref_name}')
                    )
                    after_cols = len(master_df.columns)
                    print(f"Joined {ref_name}: +{after_cols - before_cols} columns")
        
        print(f"\nFinal master dataset: {master_df.shape[0]:,} rows × {master_df.shape[1]} columns")
        print(f"Columns: {list(master_df.columns)}")
        
        # Check join quality
        print(f"\nJoin Quality Assessment:")
        join_quality = {}
        if 'UserId' in master_df.columns:
            missing_users = master_df['ContinentId'].isnull().sum()
            join_quality['User Geography'] = f"{((len(master_df) - missing_users) / len(master_df)) * 100:.1f}%"
        
        if 'AttractionId' in master_df.columns:
            missing_attractions = master_df['Attraction'].isnull().sum()
            join_quality['Attraction Info'] = f"{((len(master_df) - missing_attractions) / len(master_df)) * 100:.1f}%"
        
        for metric, score in join_quality.items():
            print(f"  {metric}: {score} join success rate")
    
    else:
        print("❌ Cannot create master dataset without transactions data")
        master_df = None
else:
    print("❌ Insufficient datasets loaded for integration")
    master_df = None

# =============================================================================
# SECTION 4: FEATURE ENGINEERING
# =============================================================================

if master_df is not None:
    print("\n4. FEATURE ENGINEERING")
    print("-" * 50)
    
    print("Creating derived features...")
    
    # Temporal features
    if 'VisitMonth' in master_df.columns:
        # Season mapping
        season_map = {12: 'Winter', 1: 'Winter', 2: 'Winter',
                     3: 'Spring', 4: 'Spring', 5: 'Spring',
                     6: 'Summer', 7: 'Summer', 8: 'Summer',
                     9: 'Fall', 10: 'Fall', 11: 'Fall'}
        master_df['Season'] = master_df['VisitMonth'].map(season_map)
        
        # Month categories
        master_df['MonthCategory'] = master_df['VisitMonth'].apply(
            lambda x: 'Peak' if x in [6, 7, 8, 12] else 'Regular'
        )
        print("✓ Created temporal features (Season, MonthCategory)")
    
    # User-level aggregated features
    if 'UserId' in master_df.columns:
        user_stats = master_df.groupby('UserId').agg({
            'Rating': ['count', 'mean', 'std'],
            'AttractionId': 'nunique'
        }).round(2)
        
        user_stats.columns = ['UserVisitCount', 'UserAvgRating', 'UserRatingStd', 'UserUniqueAttractions']
        user_stats['UserRatingStd'] = user_stats['UserRatingStd'].fillna(0)  # Single visits have no std
        
        master_df = master_df.merge(user_stats, on='UserId', how='left')
        print("✓ Created user aggregation features")
    
    # Attraction-level features
    if 'AttractionId' in master_df.columns:
        attraction_stats = master_df.groupby('AttractionId').agg({
            'Rating': ['count', 'mean', 'std'],
            'UserId': 'nunique'
        }).round(2)
        
        attraction_stats.columns = ['AttractionPopularity', 'AttractionAvgRating', 'AttractionRatingStd', 'AttractionUniqueVisitors']
        attraction_stats['AttractionRatingStd'] = attraction_stats['AttractionRatingStd'].fillna(0)
        
        master_df = master_df.merge(attraction_stats, on='AttractionId', how='left')
        print("✓ Created attraction aggregation features")
    
    # Geographic diversity features
    if all(col in master_df.columns for col in ['UserId', 'CountryId']):
        user_country_diversity = master_df.groupby('UserId')['CountryId'].nunique()
        user_country_diversity.name = 'UserCountryDiversity'
        master_df = master_df.merge(user_country_diversity, on='UserId', how='left')
        print("✓ Created geographic diversity features")
    
    # Rating deviation features
    if 'Rating' in master_df.columns and 'UserAvgRating' in master_df.columns:
        master_df['RatingDeviation'] = master_df['Rating'] - master_df['UserAvgRating']
        print("✓ Created rating deviation features")
    
    # Visit mode preferences
    if all(col in master_df.columns for col in ['UserId', 'VisitMode']):
        user_mode_counts = master_df.groupby(['UserId', 'VisitMode']).size().unstack(fill_value=0)
        user_mode_counts.columns = [f'UserMode_{col}' for col in user_mode_counts.columns]
        master_df = master_df.merge(user_mode_counts, on='UserId', how='left')
        print("✓ Created visit mode preference features")
    
    print(f"\nFeature engineering complete. Dataset now has {master_df.shape[1]} columns")

# =============================================================================
# SECTION 5: CATEGORICAL ENCODING
# =============================================================================

if master_df is not None:
    print("\n5. CATEGORICAL ENCODING")
    print("-" * 50)
    
    # Identify categorical columns
    categorical_columns = master_df.select_dtypes(include=['object']).columns.tolist()
    
    # Remove columns that are just names/addresses (not for modeling)
    exclude_columns = ['Attraction', 'AttractionAddress', 'CityName']
    categorical_columns = [col for col in categorical_columns if col not in exclude_columns]
    
    print(f"Categorical columns to encode: {categorical_columns}")
    
    # Label encoding for ordinal variables
    from sklearn.preprocessing import LabelEncoder
    
    label_encoders = {}
    for col in categorical_columns:
        if col in master_df.columns:
            le = LabelEncoder()
            master_df[f'{col}_Encoded'] = le.fit_transform(master_df[col].astype(str))
            label_encoders[col] = le
            print(f"✓ Label encoded {col}: {len(le.classes_)} unique values")
    
    # One-hot encoding for key categorical variables (limit to prevent explosion)
    onehot_columns = ['Season', 'MonthCategory']
    onehot_columns = [col for col in onehot_columns if col in master_df.columns]
    
    if onehot_columns:
        master_df_encoded = pd.get_dummies(master_df, columns=onehot_columns, prefix=onehot_columns)
        print(f"✓ One-hot encoded: {onehot_columns}")
        print(f"Dataset shape after encoding: {master_df_encoded.shape}")
    else:
        master_df_encoded = master_df.copy()
    
    # Save encoding information for later use
    encoding_info = {
        'label_encoders': label_encoders,
        'categorical_columns': categorical_columns,
        'onehot_columns': onehot_columns
    }

# =============================================================================
# SECTION 6: DATA QUALITY VALIDATION
# =============================================================================

if master_df is not None:
    print("\n6. FINAL DATA QUALITY VALIDATION")
    print("-" * 50)
    
    # Check for missing values
    missing_vals = master_df_encoded.isnull().sum()
    missing_cols = missing_vals[missing_vals > 0]
    
    if len(missing_cols) > 0:
        print("Remaining missing values:")
        for col, count in missing_cols.items():
            pct = (count / len(master_df_encoded)) * 100
            print(f"  {col}: {count} ({pct:.1f}%)")
    else:
        print("✓ No missing values in final dataset")
    
    # Check data types
    print(f"\nData type summary:")
    dtype_counts = master_df_encoded.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"  {dtype}: {count} columns")
    
    # Memory usage
    memory_mb = master_df_encoded.memory_usage(deep=True).sum() / 1024**2
    print(f"\nMemory usage: {memory_mb:.2f} MB")
    
    # Basic statistics for key columns
    if 'Rating' in master_df_encoded.columns:
        print(f"\nKey statistics:")
        print(f"  Rating range: {master_df_encoded['Rating'].min()} - {master_df_encoded['Rating'].max()}")
        print(f"  Average rating: {master_df_encoded['Rating'].mean():.2f}")
        print(f"  Total records: {len(master_df_encoded):,}")

# =============================================================================
# SECTION 7: DATA SPLITTING FOR MODELING
# =============================================================================

if master_df is not None:
    print("\n7. DATA SPLITTING FOR MODELING")
    print("-" * 50)
    
    from sklearn.model_selection import train_test_split
    
    # Define feature columns (exclude IDs and target variables)
    exclude_cols = ['TransactionId', 'UserId', 'AttractionId', 'Attraction', 'AttractionAddress']
    
    # Features for regression (predict Rating)
    regression_features = [col for col in master_df_encoded.columns 
                          if col not in exclude_cols + ['Rating']]
    
    # Features for classification (predict VisitMode)
    classification_features = [col for col in master_df_encoded.columns 
                              if col not in exclude_cols + ['VisitMode', 'VisitModeId']]
    
    if 'Rating' in master_df_encoded.columns:
        # Regression data split
        X_reg = master_df_encoded[regression_features]
        y_reg = master_df_encoded['Rating']
        
        X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
            X_reg, y_reg, test_size=0.2, random_state=42, stratify=y_reg
        )
        
        print(f"Regression split:")
        print(f"  Training: {X_reg_train.shape}")
        print(f"  Testing: {X_reg_test.shape}")
        print(f"  Features: {len(regression_features)}")
    
    if 'VisitMode' in master_df_encoded.columns:
        # Classification data split
        X_class = master_df_encoded[classification_features]
        y_class = master_df_encoded['VisitMode']
        
        X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(
            X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
        )
        
        print(f"Classification split:")
        print(f"  Training: {X_class_train.shape}")
        print(f"  Testing: {X_class_test.shape}")
        print(f"  Features: {len(classification_features)}")
    
    # Recommendation data (user-item matrix)
    if all(col in master_df_encoded.columns for col in ['UserId', 'AttractionId', 'Rating']):
        recommendation_data = master_df_encoded[['UserId', 'AttractionId', 'Rating']].copy()
        
        # Create user-item matrix
        user_item_matrix = recommendation_data.pivot_table(
            index='UserId', 
            columns='AttractionId', 
            values='Rating', 
            fill_value=0
        )
        
        print(f"Recommendation matrix: {user_item_matrix.shape}")
        print(f"  Sparsity: {(user_item_matrix == 0).sum().sum() / user_item_matrix.size * 100:.1f}%")

# =============================================================================
# SECTION 8: SAVE PROCESSED DATA
# =============================================================================

if master_df is not None:
    print("\n8. SAVING PROCESSED DATA")
    print("-" * 50)
    
    # Create processed data directory
    processed_dir = correct_path.replace('raw', 'processed')
    os.makedirs(processed_dir, exist_ok=True)
    
    # Save master dataset
    master_df_encoded.to_csv(processed_dir + 'master_dataset.csv', index=False)
    print(f"✓ Saved master dataset: {processed_dir}master_dataset.csv")
    
    # Save splits for modeling
    if 'Rating' in master_df_encoded.columns:
        pd.concat([X_reg_train, y_reg_train], axis=1).to_csv(
            processed_dir + 'train_regression.csv', index=False)
        pd.concat([X_reg_test, y_reg_test], axis=1).to_csv(
            processed_dir + 'test_regression.csv', index=False)
        print("✓ Saved regression train/test splits")
    
    if 'VisitMode' in master_df_encoded.columns:
        pd.concat([X_class_train, y_class_train], axis=1).to_csv(
            processed_dir + 'train_classification.csv', index=False)
        pd.concat([X_class_test, y_class_test], axis=1).to_csv(
            processed_dir + 'test_classification.csv', index=False)
        print("✓ Saved classification train/test splits")
    
    if 'user_item_matrix' in locals():
        user_item_matrix.to_csv(processed_dir + 'user_item_matrix.csv')
        recommendation_data.to_csv(processed_dir + 'recommendation_data.csv', index=False)
        print("✓ Saved recommendation data")
    
    # Save metadata
    import json
    metadata = {
        'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'original_shape': list(datasets['transactions'].shape) if 'transactions' in datasets else None,
        'final_shape': list(master_df_encoded.shape),
        'total_features': len(regression_features) if 'regression_features' in locals() else None,
        'categorical_columns': categorical_columns if 'categorical_columns' in locals() else [],
        'data_quality_score': 'Good' if len(missing_cols) == 0 else 'Moderate'
    }
    
    with open(processed_dir + 'processing_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print("✓ Saved processing metadata")

# =============================================================================
# SECTION 9: SUMMARY AND NEXT STEPS
# =============================================================================

print("\n9. DATA PREPROCESSING SUMMARY")
print("-" * 50)

if master_df is not None:
    print("✅ DATA PREPROCESSING COMPLETED SUCCESSFULLY")
    print(f"   Original data: {len(datasets['transactions']):,} transactions" if 'transactions' in datasets else "")
    print(f"   Final dataset: {master_df_encoded.shape[0]:,} rows × {master_df_encoded.shape[1]} columns")
    print(f"   Memory usage: {master_df_encoded.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   Data quality: {'✓ Excellent' if len(missing_cols) == 0 else '⚠ Good'}")
    
    print(f"\n📋 Ready for modeling:")
    print(f"   • Regression (Rating prediction): {len(regression_features) if 'regression_features' in locals() else 0} features")
    print(f"   • Classification (Visit mode): {len(classification_features) if 'classification_features' in locals() else 0} features")
    print(f"   • Recommendation system: User-item matrix ready")
    
    print(f"\n📁 Processed files saved to: {processed_dir}")
    print(f"   • master_dataset.csv")
    print(f"   • train/test splits for regression and classification")
    print(f"   • recommendation data and user-item matrix")
    print(f"   • processing metadata")
    
else:
    print("❌ DATA PREPROCESSING FAILED")
    print("Please check data loading and try again")

print("\n" + "=" * 80)
print("🎯 NEXT STEP: Feature Engineering and Model Building")
print("=" * 80)

TOURISM ANALYTICS - DATA CLEANING AND PREPROCESSING

1. LOADING DATA WITH AUTO-DETECTION
--------------------------------------------------
Current working directory: d:\Tourism-Analytics\notebooks
Using data path: ../data/raw/
✓ transactions: 52,930 rows
✓ users: 33,530 rows
✓ items: 1,698 rows
✓ cities: 9,143 rows
✓ countries: 165 rows
✓ regions: 22 rows
✓ continents: 6 rows
✓ attraction_types: 17 rows
✓ visit_modes: 6 rows

Loaded 9 datasets successfully

2. DATA QUALITY ASSESSMENT AND CLEANING
--------------------------------------------------

📊 CLEANING TRANSACTIONS
----------------------------------------
Shape change: (52930, 7) → (14185, 7)
Cleaning actions taken:
  • Removed 38745 rows with invalid VisitYear values

📊 CLEANING USERS
----------------------------------------
Missing values found:
  CityId: 4 (0.0%)
Shape change: (33530, 5) → (33530, 5)
Cleaning actions taken:
  • Filled CityId missing values with median

📊 CLEANING ITEMS
----------------------------------------